# Natural Language Processing

# Agents
- The core idea of agents is to use a language model to choose `a sequence of actions` to take. 
- In chains, a sequence of actions is hardcoded (in code). 
- In agents, a language model is used as a reasoning engine to determine `which actions to take and in which order.`

**LLM**: The language model powering the agent.

**Tools**: A function that performs a specific duty. This can be things like: Google Search, Database lookup, Python REPL, other chains. The interface for a tool is currently a function that is expected to have a string as an input, with a string as an output.

**Agent**: The agent to use. This should be a string that references a support agent class. Because this notebook focuses on the simplest, highest level API, this only covers using the standard supported agents. If you want to implement a custom agent, see the documentation for custom agents (coming soon).


In [2]:
# !pip3 install langchain-community
# !pip3 install langchain-core
# !pip3 install langchain-experimental
# !pip3 install langchain==0.0.354
# !pip3 install langchainhub

In [3]:
import langchain
langchain.__version__

'0.0.354'

In [4]:
import os
import torch
# Set GPU device
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

os.environ['http_proxy']  = 'http://192.41.170.23:3128'
os.environ['https_proxy'] = 'http://192.41.170.23:3128'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

## 1. LLM

In [11]:
import torch
from tqdm.auto import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import AutoTokenizer, pipeline, AutoModelForSeq2SeqLM
from transformers import BitsAndBytesConfig
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

model_id = "models/fastchat-t5-3b-v1.0"
# model_id = "models/TinyLlama-1.1B-Chat-v1.0"

tokenizer = AutoTokenizer.from_pretrained(
    model_id,
)
tokenizer.pad_token_id = tokenizer.eos_token_id

bitsandbyte_config = BitsAndBytesConfig(
        load_in_4bit = True,
        bnb_4bit_quant_type = "nf4",
        bnb_4bit_compute_dtype = torch.float16,
        bnb_4bit_use_double_quant = True
    )

model = AutoModelForSeq2SeqLM.from_pretrained(
    model_id,
    quantization_config = bitsandbyte_config,
    device_map = 'auto',
    load_in_8bit = True,
)

pipe = pipeline(
    task="text2text-generation",
    # task="text-generation",
    model=model, 
    tokenizer=tokenizer, 
    max_new_tokens=256, 
    model_kwargs = { 
        "temperature":0, 
        "repetition_penalty": 1.5
    }, 
)

llm = HuggingFacePipeline(pipeline = pipe)


## 2. Tools
Tools are functions that an agent can invoke. There are two important design considerations around tools:
1. Giving the agent access to the right tools
2. Describing the tools in a way that is most helpful to the agent
   
Without thinking through both, you won't be able to build a working agent. If you don't give the agent access to a correct set of tools, it will never be able to accomplish the objectives you give it. If you don't describe the tools well, the agent won't know how to use them properly.

LangChain provides a wide set of built-in tools, but also makes it easy to define your own (including custom descriptions). [Explore the tools integrations section](https://python.langchain.com/docs/integrations/tools/)

### Retriever Tool
Now we need to create a tool for our retriever. The main things we need to pass in are a name for the retriever as well as a description. These will both be used by the language model, so they should be informative.

In [6]:
from langchain.document_loaders import PyMuPDFLoader

nlp_document = './docs/pdf/SpeechandLanguageProcessing_3rd_07jan2023.pdf'
loader = PyMuPDFLoader(nlp_document)
documents = loader.load()
# len(documents)

from langchain.text_splitter import RecursiveCharacterTextSplitter
import torch
from langchain_community.embeddings import HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 700,
    chunk_overlap = 100
)
docs = text_splitter.split_documents(documents) 
# len(docs)

model_name = 'hkunlp/instructor-base'

embedding_model = HuggingFaceInstructEmbeddings(
    model_name = model_name,              
    model_kwargs = {
        'device': torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    },
)

vectordb = FAISS.from_documents(docs, embedding_model)
retriever = vectordb.as_retriever()

load INSTRUCTOR_Transformer
max_seq_length  512


In [7]:
from langchain.agents.agent_toolkits import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever = retriever,
    name = "search_nlp_stanford",
    description = "Searches and returns documents regarding the nlp-stanford.",
)

retriever_tool

Tool(name='search_nlp_stanford', description='Searches and returns documents regarding the nlp-stanford.', args_schema=<class 'langchain.tools.retriever.RetrieverInput'>, func=<bound method BaseRetriever.get_relevant_documents of VectorStoreRetriever(tags=['FAISS', 'HuggingFaceInstructEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7f6eb82f7d60>)>, coroutine=<bound method BaseRetriever.aget_relevant_documents of VectorStoreRetriever(tags=['FAISS', 'HuggingFaceInstructEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7f6eb82f7d60>)>)

In [8]:
retriever_tool('Transformers')

[Document(page_content='formers are not based on recurrent connections (which can be hard to parallelize),\nwhich means that transformers can be more efﬁcient to implement at scale.\nTransformers map sequences of input vectors (x1,...,xn) to sequences of output\nvectors (y1,...,yn) of the same length. Transformers are made up of stacks of trans-\nformer blocks, each of which is a multilayer network made by combining simple\nlinear layers, feedforward networks, and self-attention layers, the key innovation of\nself-attention\ntransformers. Self-attention allows a network to directly extract and use information\nfrom arbitrarily large contexts without the need to pass it through intermediate re-', metadata={'source': './docs/pdf/SpeechandLanguageProcessing_3rd_07jan2023.pdf', 'file_path': './docs/pdf/SpeechandLanguageProcessing_3rd_07jan2023.pdf', 'page': 219, 'total_pages': 636, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'LaTeX with hyperre

In [9]:
# #### alternative ways
# from langchain.agents import Tool
# from langchain.chains import RetrievalQA

# knowledge_base = RetrievalQA.from_chain_type(
#     llm=llm, chain_type="stuff", retriever=retriever, verbose = True
# )

# retriever_tool = Tool(
#     name="search_nlp_stanford",
#     func=knowledge_base.run,
#     description="Searches and returns documents regarding the nlp-stanford."
# )

In [10]:
# retriever_tool('Transformers')

### Search Tool
This notebook shows off usage of various search tools.
[Explore more tools](https://python.langchain.com/docs/integrations/tools)

[get SERPAPI_API_KEY click](https://serpapi.com/)

In [11]:
# !pip3 install google-search-results

In [12]:
os.environ['SERPAPI_API_KEY'] = "6068c2ee7f7da49ffbbd077d9847f80afe5c8e946c9d959aec66fc7e7b944eab" 

In [13]:
from langchain.agents.load_tools import load_tools
tool_names = ["serpapi"]

list_load_tools = load_tools(tool_names)
list_load_tools

[Tool(name='Search', description='A search engine. Useful for when you need to answer questions about current events. Input should be a search query.', func=<bound method SerpAPIWrapper.run of SerpAPIWrapper(search_engine=<class 'serpapi.google_search.GoogleSearch'>, params={'engine': 'google', 'google_domain': 'google.com', 'gl': 'us', 'hl': 'en'}, serpapi_api_key='6068c2ee7f7da49ffbbd077d9847f80afe5c8e946c9d959aec66fc7e7b944eab', aiosession=None)>, coroutine=<bound method SerpAPIWrapper.arun of SerpAPIWrapper(search_engine=<class 'serpapi.google_search.GoogleSearch'>, params={'engine': 'google', 'google_domain': 'google.com', 'gl': 'us', 'hl': 'en'}, serpapi_api_key='6068c2ee7f7da49ffbbd077d9847f80afe5c8e946c9d959aec66fc7e7b944eab', aiosession=None)>)]

In [14]:
google_search_tools = list_load_tools[0]
google_search_tools

Tool(name='Search', description='A search engine. Useful for when you need to answer questions about current events. Input should be a search query.', func=<bound method SerpAPIWrapper.run of SerpAPIWrapper(search_engine=<class 'serpapi.google_search.GoogleSearch'>, params={'engine': 'google', 'google_domain': 'google.com', 'gl': 'us', 'hl': 'en'}, serpapi_api_key='6068c2ee7f7da49ffbbd077d9847f80afe5c8e946c9d959aec66fc7e7b944eab', aiosession=None)>, coroutine=<bound method SerpAPIWrapper.arun of SerpAPIWrapper(search_engine=<class 'serpapi.google_search.GoogleSearch'>, params={'engine': 'google', 'google_domain': 'google.com', 'gl': 'us', 'hl': 'en'}, serpapi_api_key='6068c2ee7f7da49ffbbd077d9847f80afe5c8e946c9d959aec66fc7e7b944eab', aiosession=None)>)

In [15]:
google_search_tools('Gemini')

'["Gemini is the third astrological sign in the zodiac. Under the tropical zodiac, the sun transits this sign between about May 21 to June 21. Gemini is represented by the twins, Castor and Pollux, known as the Dioscuri in Greek mythology. It\'s known as a positive, mutable sign.", \'Gemini type: Astrological sign.\', \'Gemini symbol: Twin.\', \'Gemini element: Air.\', \'Gemini ruling_planet: Mercury.\', \'Gemini zodiac_color: yellow.\', \'Gemini constellation: Gemini.\', \'Gemini duration_tropical_western: May 20 – June 20 (2024, UT1).\', \'Gemini fall: South Node.\', \'The secure way to buy, sell, store, and convert crypto. Millions use Gemini to diversify their portfolios. Get started.\', \'Gemini is our most capable and general model, built to be multimodal and optimized for three different sizes: Ultra, Pro and Nano.\', \'Gemini (♊︎) is the third astrological sign in the zodiac. Under the tropical zodiac, the sun transits this sign between about May 21 to June 21. Gemini is ...\',

### Calculator Tool

In [16]:
from langchain.chains import LLMMathChain
from langchain.agents import Tool

calculator = LLMMathChain.from_llm(llm=llm, verbose=True)

calculator_tool = Tool(
    name="Calculator",
    func=calculator.run,
    description="useful for when you need to answer questions about math",
)
calculator_tool

Tool(name='Calculator', description='useful for when you need to answer questions about math', func=<bound method Chain.run of LLMMathChain(verbose=True, llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['question'], template='Translate a math problem into a expression that can be executed using Python\'s numexpr library. Use the output of running this code to answer the question.\n\nQuestion: ${{Question with math problem.}}\n```text\n${{single line mathematical expression that solves the problem}}\n```\n...numexpr.evaluate(text)...\n```output\n${{Output of running the code}}\n```\nAnswer: ${{Answer}}\n\nBegin.\n\nQuestion: What is 37593 * 67?\n```text\n37593 * 67\n```\n...numexpr.evaluate("37593 * 67")...\n```output\n2518731\n```\nAnswer: 2518731\n\nQuestion: 37593^(1/5)\n```text\n37593**(1/5)\n```\n...numexpr.evaluate("37593**(1/5)")...\n```output\n8.222831614237718\n```\nAnswer: 8.222831614237718\n\nQuestion: {question}\n'), llm=HuggingFacePipeline(pipeline=<transformers.pi

### SQL Tool

In [17]:
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
from langchain.agents import Tool

database = "sqlite:///docs/sql/Chinook.db"

db = SQLDatabase.from_uri(database)

db_chain = SQLDatabaseChain.from_llm(
    llm = llm, 
    db = db, 
    verbose=True,
)
    
db_tool = Tool(
    name = 'ChinookSearch',
    func = db_chain.run,
    description="useful for when you need to find chinook database",
)

## 3. Agent
This is the chain responsible for deciding what step to take next, which powered by a language model and a prompt. The inputs to this chain are:

- `Tools`: Descriptions of available tools
- `User input`: The high level objective
- `Intermediate steps`: Any (action, tool output) pairs previously executed in order to achieve the user input

The output is the next action(s) to take or the final response to send to the user (AgentActions or AgentFinish). An action specifies a tool and the input to that tool.

Different agents have different prompting styles for reasoning, different ways of encoding inputs, and different ways of parsing the output. [Explore Agent Types here](https://python.langchain.com/docs/modules/agents/agent_types/)

In [18]:
tools = [retriever_tool, google_search_tools, db_tool]
tools

[Tool(name='search_nlp_stanford', description='Searches and returns documents regarding the nlp-stanford.', args_schema=<class 'langchain.tools.retriever.RetrieverInput'>, func=<bound method BaseRetriever.get_relevant_documents of VectorStoreRetriever(tags=['FAISS', 'HuggingFaceInstructEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7f6eb82f7d60>)>, coroutine=<bound method BaseRetriever.aget_relevant_documents of VectorStoreRetriever(tags=['FAISS', 'HuggingFaceInstructEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7f6eb82f7d60>)>),
 Tool(name='Search', description='A search engine. Useful for when you need to answer questions about current events. Input should be a search query.', func=<bound method SerpAPIWrapper.run of SerpAPIWrapper(search_engine=<class 'serpapi.google_search.GoogleSearch'>, params={'engine': 'google', 'google_domain': 'google.com', 'gl': 'us', 'hl': 'en'}, serpapi_api_key='6068c2ee7f7da49ffbbd

In [19]:
for tool in tools:
    print (tool.name)
    print (tool.description)
    print ("="*150)

search_nlp_stanford
Searches and returns documents regarding the nlp-stanford.
Search
A search engine. Useful for when you need to answer questions about current events. Input should be a search query.
ChinookSearch
useful for when you need to find chinook database


In [20]:
# from langchain.prompts import MessagesPlaceholder
# from langchain_core.messages import SystemMessage
# from langchain.agents.openai_functions_agent.base import OpenAIFunctionsAgent

# system_message = SystemMessage(
#     content=(
#     """I'm your friendly NLP chatbot named ChakyBot, here to assist Chaky and Gun with any questions they have about Natural Language Processing (NLP). 
#     If you're curious about how probability works in the context of NLP, feel free to ask any questions you may have. 
#     Whether it's about probabilistic models, language models, or any other related topic, 
#     I'm here to help break down complex concepts into easy-to-understand explanations.
#     Just let me know what you're wondering about, and I'll do my best to guide you through it!"""
#     )
# )

# prompt = OpenAIFunctionsAgent.create_prompt(
#     system_message=system_message,
# )

# prompt

In [21]:
# from langchain.agents.openai_functions_agent.base import OpenAIFunctionsAgent
# from langchain.agents import AgentExecutor

# # We will use the OpenAIFunctionsAgent
# agent = OpenAIFunctionsAgent(
#     llm=llm, 
#     tools=tools, 
#     prompt=prompt,
#     verbose=True,
# )

# agent_executor = AgentExecutor(
#     agent=agent,
#     tools=tools,
#     memory=memory,
#     verbose=True,
#     return_intermediate_steps=True,
# )

# # Importantly, we pass in return_intermediate_steps=True since we are recording that with our memory object

In [22]:
# result = agent_executor({"input": "Who are you by the way?"})
# result

In [23]:
# result = agent_executor({"input": "What is transformers?"})
# result

In [2]:
# from langchain import hub

# # Get the prompt to use - you can modify this!
# prompt = hub.pull("hwchase17/openai-functions-agent")

# from langchain.agents import create_openai_functions_agent

# agent = create_openai_functions_agent(
#     llm, 
#     tools, 
#     prompt
# )

In [1]:
# from langchain.agents import AgentExecutor

# agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# agent_executor.invoke({"input": "hi!"})